# Scoring your trained model

In the cell below, please load your model into `model`. Also if you used an image size for your input images that *isn't* 224x224, you'll need to set `image_size` to the size you used. The scoring code assumes square input images.

For example, this is how I loaded in my checkpoint:

```python
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import models

class FFClassifier(nn.Module):
    
    def __init__(self, in_features, hidden_features, 
                       out_features, drop_prob=0.1):
        super().__init__()
        
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(p=drop_prob)
        
    def forward(self, x):
        x = self.drop(F.relu(self.fc1(x)))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

    
def load_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    
    model = models.vgg16(pretrained=False)
    for param in model.parameters():
        param.requires_grad = False

    # Put the classifier on the pretrained network
    model.classifier = FFClassifier(25088, checkpoint['hidden'], 102)
    
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

model = load_checkpoint('/home/workspace/classifier.pt')
```

Your exact code here will depend on how you defined your network in the project. Make sure you use the absolute path to your checkpoint which should have been uploaded to the `/home/workspace` directory.

Run the cell, then after loading the data, press "Test Code" below. This can take a few minutes or more depending on the size of your network. Your model needs  to reach **at least 20% accuracy** on the test set to be recorded.

In [16]:
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from PIL import Image
import numpy as np
from torchvision.transforms import ToTensor
from collections import OrderedDict
import json

In [17]:
def set_model(hidden_layers,architecture):
    
    if architecture == 'resnet18':
        model = models.resnet18(pretrained=True)
        input_size = model.fc.in_features
    elif architecture == 'alexnet':
        model = models.alexnet(pretrained=True)
        input_size = model.classifier[1].in_features
    elif architecture == 'densenet161':
        model = models.densenet161(pretrained=True)
        input_size = model.classifier.in_features
    elif architecture == 'inception_v3':
        model = models.inception_v3(pretrained=True)
        input_size = model.fc.in_features
    else:
        model = models.vgg16(pretrained=True)
        input_size = model.classifier[0].in_features

    #model = models.vgg16(pretrained=True)
    #print(model)

    for param in model.parameters():
        param.requires_grad = False



    classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(input_size, 4096)),
                          ('relu1', nn.ReLU()),
                          ('dropout1',nn.Dropout(p=0.2)),
                          ('fc2', nn.Linear(4096, hidden_layers)),
                          ('relu2', nn.ReLU()),
                          ('dropout2',nn.Dropout(p=0.2)),
                          ('fc3', nn.Linear(hidden_layers, 392)),
                          ('relu3', nn.ReLU()),
                          ('dropout3',nn.Dropout(p=0.2)),
                          ('fc4', nn.Linear(392, 102)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
    if architecture == 'resnet18' or architecture == 'inception_v3':
        model.fc = classifier
    else:
        model.classifier = classifier
    
    return model

In [20]:
def load_checkpoint(filepath):
    #checkpoint = torch.load(filepath)
    checkpoint = torch.load(filepath,map_location='cpu')
    model = set_model(checkpoint['hidden units'],checkpoint['architecture'])
    model.load_state_dict(checkpoint['state_dict'],strict=False)
    model.class_to_idx = checkpoint['mapping']
    #optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return model

In [21]:
# Load your model to this variable
model = load_checkpoint('/home/workspace/checkpoint.pth')
   
# If you used something other than 224x224 cropped images, set the correct size here
image_size = 224
# Values you used for normalizing the images. Default here are for 
# pretrained models from torchvision.
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
